# Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator <br>-- Correctness Testing --

## This operator is contributed by Chevron Energy Technology Company (2020)

This operator is based on simplfications of the systems presented in:
<br>**Self-adjoint, energy-conserving second-order pseudoacoustic systems for VTI and TTI media for reverse time migration and full-waveform inversion** (2016)
<br>Kenneth Bube, John Washbourne, Raymond Ergas, and Tamas Nemeth
<br>SEG Technical Program Expanded Abstracts
<br>https://library.seg.org/doi/10.1190/segam2016-13878451.1

## Introduction 

The goal of this tutorial set is to generate and prove correctness of modeling and inversion capability in Devito for variable density visco- acoustics using an energy conserving form of the wave equation. We describe how the linearization of the energy conserving *skew self adjoint* system with respect to modeling parameters allows using the same modeling system for all nonlinear and linearized forward and adjoint finite difference evolutions. 

There are three notebooks in this series:

1. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Nonlinear Ops*
[ssa_01_iso_implementation1.ipynb](ssa_01_iso_implementation1.ipynb)
<br>Implement the nonlinear modeling operations. 

2. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Linearized Ops*
[ssa_02_iso_implementation2.ipynb](ssa_02_iso_implementation2.ipynb)
<br>Implement the linearized (Jacobian) ```forward``` and ```adjoint``` modeling operations.

3. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Correctness Testing*
[ssa_03_iso_correctness.ipynb](ssa_03_iso_correctness.ipynb)
<br>Tests the correctness of the implemented operators.

There are similar series of notebooks implementing and testing operators for VTI and TTI anisotropy ([README.md](README.md)).

Below we describe a suite of unit tests that prove correctness for our *skew self adjoint* operators.

## Outline 
1. Define symbols [[link]](#c_symbols) 
2. Definition of correctness tests [[link]](#c_howto) 
1. Analytic response in the far field [[link]]("#c_analytic")
2. Modeling operator linearity test, with respect to source [[link]]("#c_F_linearity")
3. Modeling operator adjoint test, with respect to source [[link]]("#c_F_adjoint")
4. Nonlinear operator linearization test, with respect to model [[link]]("#c_F_linearization")
5. Jacobian operator linearity test, with respect to model [[link]]("#c_J_linearity")
6. Jacobian operator adjoint test, with respect to model [[link]]("#c_J_adjoint")


<a id="c_symbols"></a>
## Table of symbols

Note we are only showing the symbols here relevant to the correctness tests, for more detail on notations see the implementation notebooks that precede this one.

| Symbol &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | Description  | Dimensionality | 
|:---|:---|:---|
| $m(x,y,z)$ | Total P wave velocity ($m_0+\delta m$) | function of space |
| $m_0(x,y,z)$ | Reference P wave velocity    | function of space |
| $\delta m(x,y,z)$ | Perturbation to P wave velocity    | function of space |
| $u(t,x,y,z)$ | Total pressure wavefield ($u_0+\delta u$)| function of time and space |
| $u_0(t,x,y,z)$ | Reference pressure wavefield | function of time and space |
| $\delta u(t,x,y,z)$ | Perturbation to pressure wavefield | function of time and space |
| $s(t,x,y,z)$ | Source wavefield | function of time, localized in space to source location |
| $r(t,x,y,z)$ | Receiver wavefield | function of time, localized in space to receiver locations |
| $\delta r(t,x,y,z)$ | Receiver wavefield perturbation | function of time, localized in space to receiver locations |
| $A\ u$ | Wave-equation operator | Linear in $u$: $\quad A\ u = s$ |
| $A^{-1} s$ | Wave-equation solution operator | Linear in $s$: $\quad A^{-1} s = u$ |
| $F[m]\ s$ | Restricted Forward modeling operator | Nonlinear in $m$, Linear in $s$: $\quad r = P_r\ A^{-1} P_s^\top\ s$ |
| $F[m]^\top\ r$ | Restricted Adjoint modeling operator | Nonlinear in $m$, Linear in $r$: $\quad s = P_s\ A^{-\top}\ P_r^\top\ r$ |
| $P_s$, $P_s^\top$ | Source interpolation operator, adjoint | Linear in $s$: $\quad s = P_s\ A^{-\top}\ P_r^\top\ r$ |
| $P_r$, $P_r^\top$ | Receiver interpolation operator, adjoint | Linear in $r$: $\quad r = P_r\ A^{-1} P_s^\top$ |
| $F[m; s]$ | Forward nonlinear modeling operator | Nonlinear in $[m; s]$: $\quad$ maps $m \rightarrow r$ |
| $\nabla F[m; s]\ \delta m$ | Forward Jacobian modeling operator | Linearized at $[m; s]$: $\quad$ maps $\delta m \rightarrow \delta r$ |
| $\bigl( \nabla F\bigr)^\top[m; s]\ \delta r$ | Adjoint Jacobian modeling operator | Linearized at $[m; s]$: $\quad$ maps $\delta r \rightarrow \delta m$ |

<a id="c_howto"></a>
## Definition of correctness tests

We believe that if an operator passes the following suite of unit tests, it can be considered to be *righteous*.

## 1. Analytic response in the far field
Test that data generated in a wholespace matches analogous analytic data away from the near field. We re-use the material shown in the [examples/seismic/acoustic/accuracy.ipynb](https://github.com/devitocodes/devito/blob/master/examples/seismic/acoustic/accuracy.ipynb) notebook. 
<br>

## 2. Modeling operator linearity test, with respect to source
For random vectors $s$ and $r$, prove:

$$
\begin{aligned}
F[m]\ (\alpha\ s) &\approx \alpha\ F[m]\ s \\[5pt]
F[m]^\top (\alpha\ r) &\approx \alpha\ F[m]^\top r \\[5pt]
\end{aligned}
$$

<br>

## 3. Modeling operator adjoint test, with respect to source
For random vectors $s$ and $r$, prove:

$$
r \cdot F[m]\ s \approx s \cdot F[m]^\top r
$$

<br>

## 4. Nonlinear operator linearization test, with respect to model
For initial velocity model $m$ and random perturbation $\delta m$ prove that the $L_2$ norm error in the linearization $E(h)$ is second order (decreases quadratically) with the magnitude of the perturbation.

$$
E(h) = \biggl\|\ f(m+h\ \delta m) - f(m) - h\ \nabla F[m]\ \delta m\ \biggr\|
$$

One way to do this is to run a suite of $h$ values decreasing by a factor of $\gamma$, and prove the error decreases by a factor of $\gamma^2$:  

$$
\frac{E\left(h\right)}{E\left(h/\gamma\right)} \approx \gamma^2
$$

Elsewhere in Devito tutorials, this relation is proven by fitting a line to a sequence of $E(h)$ for various $h$ and showing second order error decrease.

<br>

## 5. Jacobian operator linearity test, with respect to model
For initial velocity model $m$ and random vectors $\delta m$ and $\delta r$, prove:

$$
\begin{aligned}
\nabla F[m]\ (\alpha\ \delta m) &\approx \alpha\ \nabla F[m]\ \delta m \\[5pt]
(\nabla F[m])^\top (\alpha\ \delta r) &\approx \alpha\ (\nabla F[m])^\top \delta r
\end{aligned}
$$

<br>

## 6. Jacobian operator adjoint test, with respect to model
For initial velocity model $m$ and random vectors $\delta m$ and $\delta r$, prove:

$$
\delta r \cdot \nabla F[m]\ \delta m \approx \delta m \cdot (\nabla F[m])^\top \delta r
$$

<br>

## Skew symmetry test for shifted derivatives
In addition to these tests, recall that in the first notebook ([ssa_01_iso_implementation1.ipynb](ssa_01_iso_implementation1.ipynb)) we implemented a unit test that demonstrates skew symmetry of the Devito generated shifted derivatives. We include that test in our suite of unit tests for completeness. Please see the first notebook for that demonstration, verifying the following relation:

$$
x_2 \cdot \left( \overrightarrow{\partial_x}\ x_1 \right) \approx -\ 
x_1 \cdot \left( \overleftarrow{\partial_x}\ x_2 \right) 
$$

## Implementation of correctness tests

Below we implement the correctness tests described above. These tests are copied from standalone tests that run in the Devito project *continuous integration* (CI) pipeline. We will implement the test methods in one cell and then call from the next cell to verify correctness, but note that a wider variety of parameterization is tested in the CI pipeline.

For these tests we use the convenience functions implemented in ```operators.py``` and ```wavesolver.py``` rather than implement the operators in the notebook as we have in the first two notebooks in this series. Please review the source to compare with our notebook implementations:
- [operators.py](operators.py)
- [wavesolver.py](wavesolver.py)

## Imports 

We have grouped all imports used in this notebook here for consistency.

In [1]:
import numpy as np
from examples.seismic import RickerSource, Receiver, TimeAxis
from devito import (Grid, Function, TimeFunction, SpaceDimension, Constant, 
                    Eq, Operator, solve, configuration)
from devito.finite_differences import Derivative
from devito.builtins import gaussian_smooth
from examples.seismic.skew_self_adjoint import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from timeit import default_timer as timer

# These lines force images to be displayed in the notebook, and scale up fonts 
%matplotlib inline
mpl.rc('font', size=14)

# We define 32 bit floating point as the precision type 
dtype = np.float32

# Set the default language to openmp
configuration['language'] = 'openmp'

# Set logging to debug, captures statistics on the performance of operators
configuration['log-level'] = 'DEBUG'

<a id="c_analytic"></a>
## 1. Analytic response in the far field
Test that data generated in a wholespace matches analogous analytic data away from the near field. We re-use the material shown in the [examples/seismic/acoustic/accuracy.ipynb](https://github.com/devitocodes/devito/blob/master/examples/seismic/acoustic/accuracy.ipynb) notebook. 

<a id="c_F_linearity"></a>
## 2. Modeling operator linearity test, with respect to source
For random vectors $s$ and $r$, prove:

$$
\begin{aligned}
F[m]\ (\alpha\ s) &\approx \alpha\ F[m]\ s \\[5pt]
F[m]^\top (\alpha\ r) &\approx \alpha\ F[m]^\top r \\[5pt]
\end{aligned}
$$

<br>


<a id="c_F_adjoint"></a>
## 3. Modeling operator adjoint test, with respect to source
For random vectors $s$ and $r$, prove:

$$
r \cdot F[m]\ s \approx s \cdot F[m]^\top r
$$

<br>


<a id="c_F_linearization"></a>
## 4. Nonlinear operator linearization test, with respect to model
For initial velocity model $m$ and random perturbation $\delta m$ prove that the $L_2$ norm error in the linearization $E(h)$ is second order (decreases quadratically) with the magnitude of the perturbation.

$$
E(h) = \biggl\|\ f(m+h\ \delta m) - f(m) - h\ \nabla F[m]\ \delta m\ \biggr\|
$$

One way to do this is to run a suite of $h$ values decreasing by a factor of $\gamma$, and prove the error decreases by a factor of $\gamma^2$:  

$$
\frac{E\left(h\right)}{E\left(h/\gamma\right)} \approx \gamma^2
$$

Elsewhere in Devito tutorials, this relation is proven by fitting a line to a sequence of $E(h)$ for various $h$ and showing second order error decrease.

<br>


<a id="c_J_linearity"></a>
## 5. Jacobian operator linearity test, with respect to model
For initial velocity model $m$ and random vectors $\delta m$ and $\delta r$, prove:

$$
\begin{aligned}
\nabla F[m]\ (\alpha\ \delta m) &\approx \alpha\ \nabla F[m]\ \delta m \\[5pt]
(\nabla F[m])^\top (\alpha\ \delta r) &\approx \alpha\ (\nabla F[m])^\top \delta r
\end{aligned}
$$

<br>


<a id="c_J_adjoint"></a>
## 6. Jacobian operator adjoint test, with respect to model
For initial velocity model $m$ and random vectors $\delta m$ and $\delta r$, prove:

$$
\delta r \cdot \nabla F[m]\ \delta m \approx \delta m \cdot (\nabla F[m])^\top \delta r
$$

<br>
